# Bangalore Housing Price Analysis & Prediction

<b>This is my first notebook on Kaggle!</b> I have attempted to predict house prices in Bangalore using some fairly simple techniques and a bit of feature engineering, on which I would appreciate your feedback. From time to time I will also share my understanding of price drivers, outliers and limitations which arise due the absence of certain key features. I am new to programming, although I have over 18 years of experience in different industries, a lot of them in analytical roles dealing with various kinds of data. With further notebook submissions I hope to improve the quality of code!

<h3>There is not a ton of EDA here and I have focused more on approach.</h3>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_absolute_error

In [ ]:
bangalore = pd.read_csv('../input/housing-prices-in-metropolitan-areas-of-india/Bangalore.csv')

In [ ]:
bangalore.head()

In [ ]:
bangalore.info()

In [ ]:
len(bangalore)

This dataset is already available on Kaggle in cleaned up form and we can see there are no missing values in any of the columns. (6207/6207 for all features). We also see that there are close to 40 potential features that can determine the price of the apartment. Except for 'Area', 'Location' and 'No. of Bedrooms' all other features are Boolean type, i.e. either present or not present. We do not have any ordinal features (such as good/average or old/new etc in this case). 

A key metric in the housing industry, at least in India, is <b>"Price Per Square Feet (Price/Area)"</b>, expressed as Rupees Per Square Feet (Rupee being our currency). Almost everyone uses this metric to compare the prices as it eliminates the impact of house size and provides an apple to apple comparison. Let us add a column that captures Price Per Square Feet. We will call this <b>"PricePSF"</b> in short

In [ ]:
bangalore['PricePSF'] = bangalore['Price'] / bangalore['Area']

Let us check the distribution of the non-boolean and non-string elements of our data

In [ ]:
def dist_plot (feature, color, position=121):
    plt.figure(figsize=(15,4))
    plt.subplot(position)
    sns.boxplot(x=feature, data=bangalore, color=color)
    plt.subplot(position+1)
    sns.distplot(bangalore[feature], color=color)
dist_plot('Price', 'red')
dist_plot('Area', 'grey')
dist_plot('PricePSF', 'teal')

As we can see above, the data is <b>heavily right-skewed</b> for Price and Area both. One would expect that PricePSF would be normally distributed as it is a ratio, however it is not so. This indicates either or both of the following:

1. There is much more skewness in Price compared to Area
2. The houses with high Prices are not necessarily the ones with the large Area, hence PricePSF is not able to normalize such cases

We now proceed to <b>remove outliers</b> so that they don't have a lopsided impact on our model. As the data is heavily right skewed, we remove a higher percentile (10%) from the top versus bottom(5%)

In [ ]:
# define outliers
price_psf_outliers = np.percentile(bangalore.PricePSF, [5,90])
price_outliers = np.percentile(bangalore.Price, [5,90])
area_outliers = np.percentile(bangalore.Area, [5,90])
# create filters based on outliers
price_psf_filter = (bangalore.PricePSF > price_psf_outliers[0]) & (bangalore.PricePSF < price_psf_outliers[1])
price_filter = (bangalore.Price > price_outliers[0]) & (bangalore.Price < price_outliers[1])
area_filter = (bangalore.Area > area_outliers[0]) & (bangalore.Area < area_outliers[1])
# apply filters
bangalore = bangalore[(price_psf_filter) & (price_filter) & (area_filter)]

Let us check the distribution again

In [ ]:
dist_plot('Price', 'red')
dist_plot('Area', 'grey')
dist_plot('PricePSF', 'teal')

The above distributions looks far better. Though there is still right-skewnesss, we can expect this from real estate prices as a small number of large size and expensive properties will pull the distribution to the right. 

<h3>Let us briefly discuss what drives a house price (longish read but important):</h3>

1. Area - Larger, would logically mean more expensive, all else held constant
2. Location - Factoring this requires knowledge of local area. We will try a workaround for this
3. Amenities - This is tricky. High rise apartment complexes will have lot of amenities compared to independent houses and villas. Yet they may be cheaper because all these amenities are shared. In the subsequent sections I have tried to create a 'FeatureScore' for amenities (higher FeatureScore should imply higher Price). However I admit that independent houses may lose out on this and the model may underprice such houses. I would welcome inputs on how to tackle this. 
4. Floor rise - In high rise complexes in India, houses on higher floors are priced higher by a linear formula. Floor data is absent from the data set.
5. Developer Reputation - Again this is very important in India. Two apartments in the same Location by developers of different repute have markedly different PricePSF. This information is absent from the data set. Even if it was available, one would require some local knowledge to factor in developer score. 
6. Resale or not - Another tricky one. In absence of the age of the house, only a Resale binary may not produce accurate results. A one year old resale versus 20 year old are very different. In some cases a 3-4 year resale may fetch a higher PricePSF as the complex would have matured and be niggle free, better landscaped etc.

<h3>As we can see, quite a few important variables are not available in the data set, hence our model may not perform very well on the test set</h3>

Let us now check the features available to us and also clean up the names so that they do not cause any problems later

In [ ]:
bangalore.columns

Two columns seem to have problematic names. Let's rename them

In [ ]:
bangalore.rename(columns={'No. of Bedrooms':'Bedrooms', "Children'splayarea":'PlayArea'}, inplace=True)

As we can see from the columns list, there are a lot of features, most of them binary. Most of the binary features will fit under the broad category of <b>Amenities</b>. In the following section I have tried to calculate feature scores for each house bases on the Amenities being absent or present. 

<b><u>Note: Important</u></b> - I have assigned weights to binary features based on my discretion. You are free to use the weights you want. I have also used a scale of 0-4 (0 being least important and 4 being most important). Again the choice of this scale is entirely up to you. 

In [ ]:
# assign weights to features
feature_dict = {'MaintenanceStaff':2, 'Gymnasium':4, 'SwimmingPool':4,'LandscapedGardens':3, 'JoggingTrack':3, 'RainWaterHarvesting':2,'IndoorGames':3, 'ShoppingMall':2, 'Intercom':2, 'SportsFacility':3, 'ATM':2, 'ClubHouse':2, 'School':2, '24X7Security':1, 'PowerBackup':4, 'CarParking':3, 'StaffQuarter':0, 'Cafeteria':0, 'MultipurposeRoom':2, 'Hospital':3, 'WashingMachine':0, 'Gasconnection':2, 'AC':0, 'Wifi':0, 'PlayArea':3, 'LiftAvailable':0, 'BED':0, 'VaastuCompliant':0, 'Microwave':0, 'GolfCourse':0, 'TV':0, 'DiningTable':0, 'Sofa':0, 'Wardrobe':0, 'Refrigerator':0}

In [ ]:
# convert to Dataframe
features = pd.DataFrame(feature_dict.items(), columns=['Features', 'Weight'])

In [ ]:
features.head()

In [ ]:
# The features matrix has 35 features
features.shape

To arrive at Feature Score for each house, we need to calculate dot product between the features matrix and the features binaries. Before this, we need to <b><u>subset a features matrix</u></b> from the bangalore dataset

In [ ]:
features_matrix = bangalore[['MaintenanceStaff', 'Gymnasium', 'SwimmingPool',
       'LandscapedGardens', 'JoggingTrack', 'RainWaterHarvesting',
       'IndoorGames', 'ShoppingMall', 'Intercom', 'SportsFacility', 'ATM',
       'ClubHouse', 'School', '24X7Security', 'PowerBackup', 'CarParking',
       'StaffQuarter', 'Cafeteria', 'MultipurposeRoom', 'Hospital',
       'WashingMachine', 'Gasconnection', 'AC', 'Wifi', 'PlayArea',
       'LiftAvailable', 'BED', 'VaastuCompliant', 'Microwave',
       'GolfCourse', 'TV', 'DiningTable', 'Sofa', 'Wardrobe',
       'Refrigerator']]

In [ ]:
features_matrix.tail()

Some examination (not shown here) revelas that the features matrix contains (0, 1, 9) as values. I would assume that 9 represents NaN. Let us replace all 9s by 0s

In [ ]:
features_matrix.replace(9, 0, inplace=True)

In [ ]:
features_matrix.shape

We will now compute <b>dot product</b> between the "features_matrix" <b>(shape = 4453 x 35)</b> and 'Weight' column of the "features" dataframe <b>(shape = 35 x 1)</b>. This will result in a column vector of shape <b>4453 x 1</b> and is nothing but the FeatureScore for each house. We will then join this resulting vector with the bangalore dataframe

In [ ]:
# apply dot product to compute feature score for every row
feature_scores_df = np.dot(features_matrix, features['Weight'])

In [ ]:
# join feature scores column with bangalore dataframe
bangalore['FeatureScore'] = feature_scores_df

In [ ]:
bangalore.head(3)

We have now reduced the number of features on which to run our regression model. These are:

1. Area (numeric)
2. Location (string). In next section I will propose a way to tackle this. 
3. Bedrooms (numeric)
4. Feature Score (numeric)
4. Resale (boolean)

<b>Before we proceed let us check if our numerical features have any colinearity</b>

In [ ]:
corr_df = bangalore[['Area', 'Resale', 'Bedrooms', 'FeatureScore', 'Price']]
sns.heatmap(corr_df.corr(method='pearson'), cmap='RdYlGn_r', linewidths=2)

<h3>Observations</h3>

1. We can see that <b>Area and Bedrooms are somewhat correlated</b>. This is not surprising as houses with larger area will tend to have more bedrooms and vice versa.
2. Area has a stronger correlation with Price compared to Bedrooms. If forced to make a choice, we should drop bedrooms to avoid colinearity
3. Interestingly FeatureScore does not seem to be correlated to Price. However, remember that the feature weights were my arbitrary choice and may be wrong. An industry expert can help here. For this analysis we will retain FeatureScore in our model

<h3>Let us now explore how we can model Location as a numeric feature</h3>

In [ ]:
# Create a pivot table of Locations with PricePSF as the value
location_pivot = pd.pivot_table(data=bangalore, index='Location', aggfunc='mean', values='PricePSF')

In [ ]:
location_pivot

<b>Location is one of the most important determinants of house price</b>. I tried **One Hot Encoding** to factor locations, but with disastrous results as there were too many locations with few observations. As a bit of feature engineering, I am going to calculate the 'Location Premium' for every Location. This is nothing but the PricePSF for every location divided by the minimum PricePSF. This sets the cheapest location as the base location (with a score of 1) and every other location has a premium as a multiple of that base location 

In [ ]:
location_pivot['LocationPremium'] = location_pivot['PricePSF'] / location_pivot['PricePSF'].min()

In [ ]:
location_pivot.sort_values('LocationPremium', ascending=False)

We now merge the location pivot with the bangalore dataframe on 'Location' column

In [ ]:
bangalore = pd.merge(bangalore, location_pivot['LocationPremium'], on='Location')

As a final step we calculate the <b>log</b> of LocationPremium. I observed that it yields better results than simply using LocationPremium

In [ ]:
bangalore['LogPremium'] = np.log(bangalore['LocationPremium'])

In [ ]:
# check if all required columns are present
bangalore.head(3)

<b>We are now ready to prepare our Linear Regression Model</b>

In [ ]:
x = bangalore[['Area', 'FeatureScore', 'Resale', 'LogPremium', 'Bedrooms']]
y = bangalore['Price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=19)

<b>I used Statsmodel to run the regression as it yields a useful summary which is not provided by ScikitLearn</b>

In [ ]:
model = sm.OLS(y_train, x_train).fit()
model.summary()

<h3>Observations</h3>

1. R squared value of 92.3% is good. However, we are yet to see how the model performs on test data
2. Feature Score has a p-value of greater than 0.05, indicating that it is not an important determinant of price. Either that or I have not engineered it well.
3. FeatureScore also has a very high range (even entering negative territory). Clearly there is much more to be engineered here as presence of even a single feature should not reduce the price of a house.
4. Bedrooms has a negative coefficient, indicating more bedrooms lower the price. This seems like an anomaly and is a likely result of colinearity between bedrooms and Area. If modeled in the absense of Area, Bedrooms may result in a positive correlation.
5. 'Resale = True' negatively impacts the price, which is understandable. However the extent (almost a million Rupees) may be a subject for further analysis for industry experts

<b>Let us now see how the model performs on test data</b>

In [ ]:
predictions = model.predict(x_test)
plt.scatter(x=y_test, y=predictions)
plt.xlabel('Actual Test Prices')
plt.ylabel('Predicted Test Prices')

Although there seems to be a linear correlation between test set Prices and predicted Prices, there is also a fair bit of scatter. Let us check the residuals plot

In [ ]:
residuals = y_test - predictions
sns.distplot(residuals)
plt.xlabel('Residuals')

The residuals seem to be mostly normally distributed, except for a slight skew to the right. Residuals bordering on close to Rs 10 million are problematic. Let us now check the plot of residuals against predicted prices

In [ ]:
sns.regplot(x=predictions, y=residuals)
plt.xlabel('Predicted Prices')
plt.ylabel('Residuals')

In [ ]:
mean_absolute_error(y_test,predictions)

In [ ]:
predictions.mean()

<h3>Observations</h3>

1. The residuals-predictions plot has a somewhat funnel/cone shape towards the right, indicating Heteroscadasticity. This is to be expected in housing prices as they do not vary uniformly. Houses which are very large or very small tend to be in short supply and hence command a higher PricePSF compared to commonly available sizes. 
2. For higher prices our model makes larger errors. I had expected this
3. The residuals seem to be distributed almost equally on both sides of the straight line, indicating that our model over predicts and under predicts almost equally
4. Mean absolute error of Rupees 1.65 million looks high for a range of values which has a mean of Rs 7.64 million

Let us run our model once again, this time excluding Bedrooms as a predictor. Despite evidence against FeatureScore, I am retaining it as I am not fully convinced that features don't have an impact

In [ ]:
x2 = bangalore[['Area', 'FeatureScore', 'Resale', 'LogPremium']]
y2 = bangalore['Price']
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.2, random_state=19)

In [ ]:
model2 = sm.OLS(y_train2, x_train2).fit()
model2.summary()

<h3>Observations</h3>

1. The R-squared value is almost unchanged, but what is important to see is that the coefficient for Area has reduced. I like this because to me this coefficient represents the cost of construction and it is now closer to the real value (the cost of construction in India varies from Rupees 2500-3500 per square feet). This should be similar for all houses regardless of area and the premium should actually be derived from the Location
2. The p-value of FeatureScore has increased, indicating it is even less important in this second model

<b>Let us make predictions using model2</b>

In [ ]:
predictions2 = model2.predict(x_test2)

I am skipping the plots and only concerning myself with the mean squared error for the time being

In [ ]:
mean_absolute_error(y_test2,predictions2)

In [ ]:
predictions2.mean()

<h2>Concluding Remarks</h2>

1. Model2 does not seem to be performing better than Model1 on test data.
2. However, I do believe that Model2 does better than Model1 in assigning a lower coefficient to Area for reasons explained earlier
3. Also as noted earlier, significant drivers of Price such as Developer reputation and Age of house are not available in the data set and these may impact the predictions
4. Location Premium uses PricePSF in its calculation, which in turn uses Price, which is the dependent variable. I am not sure if and how much this impacts the model. Conceptually I need to study the impact of using a dependent variable in Feature Engineering
5. Independent Houses / Villas need a separate label from high rise apartments as their pricing structure is significantly different, namely apartments have a lower component of land value in their pricing due to shared land. However in absense of such a label we have to live with ambiguity
6. Domain/Industry expertise is very important and this is what helps in fine tuning parameters as well as identifying outliers and anomalies. 

<h3>I intend to run the model with similar assumptions on the Mumbai dataset. Mumbai as a housing market has virtually no independent houses and I would love to see if Amenities (Represented by FeatureScore) plays a more important role there. Thanks for Reading :)</h3>